In [1]:
#Connect to GIS
from arcgis.gis import GIS
gis = GIS("home")

In [2]:
#Packages
import requests
import pandas as pd
import zipfile
import os
import io
from arcgis import features, GeoAccessor, GeoSeriesAccessor
from arcgis.gis import GIS
from arcgis.features.analysis import join_features
from arcgis.features import FeatureLayerCollection
from arcgis.geometry import SpatialReference, Geometry

In [3]:
directory=r'C:\Mac\Home\Documents\ArcGIS\Projects\Lab3_Notebooks\Lab3_Notebooks.gdb'

In [4]:
#NDAWN temp data
url= 'https://ndawn.ndsu.nodak.edu/table.csv?station=78&station=111&station=98&station=162&station=174&station=142&station=164&station=138&station=161&station=9&station=160&station=224&station=159&station=10&station=229&station=118&station=56&station=165&station=11&station=12&station=58&station=13&station=84&station=218&station=55&station=179&station=7&station=186&station=87&station=14&station=15&station=96&station=191&station=16&station=210&station=201&station=137&station=124&station=143&station=17&station=85&station=226&station=140&station=134&station=18&station=136&station=219&station=65&station=104&station=99&station=192&station=19&station=227&station=129&station=20&station=101&station=166&station=178&station=81&station=21&station=97&station=22&station=75&station=184&station=2&station=211&station=172&station=139&station=158&station=23&station=157&station=220&station=62&station=86&station=24&station=89&station=126&station=223&station=167&station=93&station=183&station=90&station=25&station=205&station=83&station=107&station=156&station=77&station=26&station=155&station=70&station=127&station=144&station=27&station=173&station=132&station=28&station=195&station=185&station=29&station=30&station=154&station=31&station=187&station=102&station=32&station=119&station=4&station=217&station=80&station=33&station=59&station=153&station=105&station=82&station=225&station=34&station=198&station=72&station=135&station=35&station=76&station=120&station=209&station=141&station=109&station=36&station=207&station=79&station=193&station=71&station=212&station=37&station=38&station=189&station=39&station=130&station=73&station=188&station=40&station=41&station=54&station=228&station=69&station=194&station=145&station=214&station=113&station=128&station=42&station=43&station=103&station=171&station=116&station=196&station=88&station=114&station=3&station=163&station=200&station=216&station=64&station=115&station=168&station=67&station=175&station=146&station=170&station=197&station=44&station=206&station=133&station=106&station=100&station=121&station=45&station=46&station=61&station=66&station=181&station=74&station=213&station=60&station=199&station=125&station=176&station=177&station=8&station=180&station=204&station=47&station=221&station=122&station=108&station=5&station=152&station=48&station=151&station=147&station=68&station=169&station=49&station=50&station=91&station=182&station=117&station=63&station=150&station=51&station=6&station=222&station=52&station=92&station=112&station=131&station=123&station=95&station=53&station=203&station=190&station=208&station=57&station=149&station=148&station=202&station=215&station=110&variable=ddavt&year=2024&ttype=daily&quick_pick=30_d&begin_date=2024-11-09&end_date=2024-11-09'

#NDAWN API Call
request=requests.get(url)
response=request.content
df=pd.read_csv(io.StringIO(response.decode('utf-8')), skiprows=[0,1,2,4])
df.head()
df.drop(['Avg Temp Flag'], axis=1, inplace=True)

In [6]:
#df to feature class
sdf = GeoAccessor.from_xy(df, 'Longitude', 'Latitude')
out_name = "ndawn_fc"
sdf.spatial.to_featureclass(location=f"{directory}/{out_name}")

'C:\\Mac\\Home\\Documents\\ArcGIS\\Projects\\Lab3_Notebooks\\Lab3_Notebooks.gdb\\ndawn_fc'

In [11]:
new_sedf = sdf.groupby('Station Name').agg({'Latitude': 'mean', 'Longitude': 'mean', 'Elevation': 'max', 'Year': 'max', 'Month': 'max', 'Day': 'count', 'Avg Temp': 'mean'})
new_sedf.head()

,Latitude,Longitude,Elevation,Year,Month,Day,Avg Temp
Station Name,,,,,,,
Ada,47.32119,-96.51406,910,2024,11,30,42.748633
Adams,48.49988,-98.07588,1580,2024,11,30,39.551500
Alamo,48.54652,-103.47186,2157,2024,11,30,37.910967
Alexander,47.75056,-103.73358,2202,2024,11,30,39.761567
Alvarado,48.24594,-97.02153,809,2024,11,30,41.400133


In [12]:
new_sedf = GeoAccessor.from_xy(new_sedf, 'Longitude', 'Latitude')
out_name= "ndawn_updated"
new_sedf.spatial.to_featureclass(location=f"{directory}/{out_name}")

'C:\\Mac\\Home\\Documents\\ArcGIS\\Projects\\Lab3_Notebooks\\Lab3_Notebooks.gdb\\ndawn_updated'

In [13]:
#IDW
with arcpy.EnvManager(scratchWorkspace=r"C:\Mac\Home\Documents\ArcGIS\Projects\Lab3_Notebooks\Lab3_Notebooks.gdb"):
    out_raster = arcpy.sa.Idw(
        in_point_features="ndawn_updated",
        z_field="avg_temp",
        cell_size=0.021010908,
        power=2,
        search_radius="VARIABLE 12",
        in_barrier_polyline_features=None
    )
    out_raster.save(r"C:\Mac\Home\Documents\ArcGIS\Projects\Lab3_Notebooks\Lab3_Notebooks.gdb\idw_ndawn")

In [14]:
#Ordinary Kriging
with arcpy.EnvManager(scratchWorkspace=r"C:\Mac\Home\Documents\ArcGIS\Projects\Lab3_Notebooks\Lab3_Notebooks.gdb"):
    out_surface_raster = arcpy.sa.Kriging(
        in_point_features="ndawn_updated",
        z_field="avg_temp",
        kriging_model="Spherical # # # #",
        cell_size=0.021010908,
        search_radius="VARIABLE 12",
        out_variance_prediction_raster=None
    )
    out_surface_raster.save(r"C:\Mac\Home\Documents\ArcGIS\Projects\Lab3_Notebooks\Lab3_Notebooks.gdb\kriging_ndawn")

In [15]:
#LPI
arcpy.ga.LocalPolynomialInterpolation(
    in_features="ndawn_updated",
    z_field="avg_temp",
    out_ga_layer=None,
    out_raster=r"C:\Mac\Home\Documents\ArcGIS\Projects\Lab3_Notebooks\Lab3_Notebooks.gdb\lpi_ndawn",
    cell_size=0.021010908,
    power=1,
    search_neighborhood="NBRTYPE=Standard S_MAJOR=3.42563109432527 S_MINOR=3.42563109432527 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR",
    kernel_function="EXPONENTIAL",
    bandwidth=None,
    use_condition_number="NO_USE_CONDITION_NUMBER",
    condition_number=None,
    weight_field=None,
    output_type="PREDICTION"
)

<Result ''>